## Accessing a SQLite Database with Pandas

[Pandas](https://pandas.pydata.org/), a great python pakage, provides us a cool funtion of read_sql_query to access a SQLite database, directly.

This notebook is extracted from another tutorial of [pySQLiteSWAT](https://github.com/royalosyin/pySQLiteSWAT). If you are intested in more python-based tools such as Pandas, GeoPandas, PySAL, folium, etc., you can have a look.

### 1. Load all needed libraries

In [1]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

### 2. Use pandas to read SQLite database

#### 2.1 Create a db connection

In [2]:
con = sqlite3.connect('data\\demo.db3')
cursor = con.cursor()

#### 2.2 Check table names

Here we take the **RCH** table as an example

In [3]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[]


#### 2.3 Check columns of the RCH table

In [4]:
def table_columns(db, table_name):
    curs = db.cursor()
    sql = "select * from %s where 1=0;" % table_name
    curs.execute(sql)
    return [d[0] for d in curs.description]

In [5]:
table_columns(con, 'rch')

OperationalError: no such table: rch

#### 2.4 Read specified columns of RCH table inot a pandas DataFrame
e.g., Reach ID, Year, Month and Flow-out

In [ ]:
df = pd.read_sql_query("SELECT RCH, YR, MO, FLOW_OUTcms from rch", con)
df = df.set_index(['MO'])
print(df.head(10))

#### 2.5 Finally, do not forget to close the db connection

In [ ]:
con.close()

### 3. Further process flow data with pandas

#### 3.1 Custom quarters definition

In [ ]:
quarters = {1: 'DJF', 2: 'DJF', 3: 'MAM', 4: 'MAM', 5: 'MAM', 6: 'JJA',
            7: 'JJA', 8: 'JJA', 9: 'SON', 10: 'SON', 11: 'SON', 12: 'DJF'}

#### 3.2 Carry out seasonal statistics for each reach

In [ ]:
ssndf = df.groupby(['RCH',quarters])['FLOW_OUTcms'].mean()
ssndf.head(5)

In [ ]:
ssndf = ssndf.reset_index()
ssndf.set_index(['RCH'])
ssndf.head(5)

In [ ]:
ssndf = ssndf.rename(index=str, columns={"level_1":"SSN"})
ssndf.head(5)

In [ ]:
pivoted = ssndf.pivot(index='RCH', columns='SSN', values='FLOW_OUTcms')
pivoted.head()

### 4. Visualize

Set some parameters to make figure pretty

In [ ]:
# Plot size to 15" x 7"
matplotlib.rc('figure', figsize = (15, 7))
# Font size to 14
matplotlib.rc('font', size = 14)
# Display top and right frame lines
matplotlib.rc('axes.spines', top = True, right = True)
# Remove grid lines
matplotlib.rc('axes', grid = False)
# Set backgound color to white
matplotlib.rc('axes', facecolor = 'white')

In [ ]:
pivoted.plot(kind='bar', title='Seasonal Mean Discharge from 1981 to 2010 ($m^3/s$)')

### References

Fernando Pérez and Brian E. Granger. IPython: A System for Interactive Scientific Computing, Computing in Science & Engineering, 9, 21-29 (2007), DOI:10.1109/MCSE.2007.53

John D. Hunter. Matplotlib: A 2D Graphics Environment, Computing in Science & Engineering, 9, 90-95 (2007), DOI:10.1109/MCSE.2007.55

Wes McKinney. Data Structures for Statistical Computing in Python, Proceedings of the 9th Python in Science Conference, 51-56 (2010)

https://www.sqlite.org/lang.html